In [ ]:
import sys  
sys.path.insert(0, '/content/drive/MyDrive/overoptimism')

In [ ]:
!pip install optuna

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
def objective(trial):
    seed = 0

    results = []

    n_samples = 1000
    jitter = 0.15

    min_pts = trial.suggest_int('min_pts', 2, 100)
    eps = trial.suggest_int('eps', 1, 10) * 0.1

    datasets = []
    n_centers = None
    
    for s in range(seed, seed+10):
        datasets.append(make_moons(n_samples=n_samples, noise=jitter,
            shuffle=True, random_state=s))
        
    step = 0
    for dataset in datasets: 
        run = {}
        run['run_id'] = run_id
        run['step'] = step
        run['n_samples'] = n_samples
        run['jitter'] = jitter
        scaler = StandardScaler()
        data = scaler.fit_transform(dataset[0])

        dbscan = DBSCAN(eps=eps, min_samples=min_pts).fit(data).labels_
        run['eps'] = eps
        run['min_pts'] = min_pts

        gt = dataset[1]
        results.append(AMI(dbscan, gt))
        run['DBSCAN'] = AMI(dbscan, gt)
        step += 1
        experiment.append(run)

    return np.mean(results)

In [ ]:
experiment = []
run_id = 0

In [ ]:
import optuna
import numpy as np
import pandas as pd

from sklearn.datasets import make_blobs, make_moons
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering, MeanShift, estimate_bandwidth
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_mutual_info_score as AMI
from rock import ROCK

from util import dbscan_init

import logging
import sys

In [ ]:
study_name = 'two_moons_opt'

In [ ]:
storage_name = "sqlite:///{}.db".format(study_name)
sampler = optuna.samplers.TPESampler(seed=0)
study = optuna.create_study(study_name=study_name, storage=storage_name, sampler=sampler, load_if_exists=True,  direction="maximize")

[I 2021-05-16 10:36:05,868] A new study created in RDB with name: two_moons_opt


In [ ]:
study.optimize(objective, n_trials=100)

[I 2021-05-16 10:36:07,207] Trial 0 finished with value: 0.0 and parameters: {'min_pts': 56, 'eps': 8}. Best is trial 0 with value: 0.0.
[I 2021-05-16 10:36:07,496] Trial 1 finished with value: 0.00018209042501786225 and parameters: {'min_pts': 61, 'eps': 6}. Best is trial 1 with value: 0.00018209042501786225.
[I 2021-05-16 10:36:07,762] Trial 2 finished with value: 2.6027032913109257e-16 and parameters: {'min_pts': 43, 'eps': 7}. Best is trial 1 with value: 0.00018209042501786225.
[I 2021-05-16 10:36:08,050] Trial 3 finished with value: 0.0 and parameters: {'min_pts': 45, 'eps': 9}. Best is trial 1 with value: 0.00018209042501786225.
[I 2021-05-16 10:36:08,269] Trial 4 finished with value: 0.0 and parameters: {'min_pts': 97, 'eps': 4}. Best is trial 1 with value: 0.00018209042501786225.
[I 2021-05-16 10:36:08,539] Trial 5 finished with value: 0.0009439327152317232 and parameters: {'min_pts': 80, 'eps': 6}. Best is trial 5 with value: 0.0009439327152317232.
[I 2021-05-16 10:36:08,831] 

In [ ]:
results = pd.DataFrame(experiment)

In [ ]:
results.to_csv('two_moons_opt.csv')

In [ ]:
results.iloc[80 * 10 : 81 * 10].std()['DBSCAN']

0.024435737741039746